In [2]:
# Importing libraries & building spark session
from pyspark.sql import SparkSession
import pyspark.sql as sparksql
spark = SparkSession.builder.appName('stroke').getOrCreate()
train = spark.read.csv('train_2v.csv', inferSchema=True,header=True)

In [3]:
train.head(2)

[Row(id=30669, gender='Male', age=3.0, hypertension=0, heart_disease=0, ever_married='No', work_type='children', Residence_type='Rural', avg_glucose_level=95.12, bmi=18.0, smoking_status=None, stroke=0),
 Row(id=30468, gender='Male', age=58.0, hypertension=1, heart_disease=0, ever_married='Yes', work_type='Private', Residence_type='Urban', avg_glucose_level=87.96, bmi=39.2, smoking_status='never smoked', stroke=0)]

In [4]:
# Checking table schema
train.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [6]:
# create DataFrame as a temporary view
train.createOrReplaceTempView('table')

In [7]:
# Checking table content
spark.sql("select * from table").show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|           null|     0|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|     0|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|           null|     0|
|56543|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|     0|
|46136|  Male|14.0|           0|            0|         

In [8]:
# Checking count of people with stroke
train.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|  783|
|     0|42617|
+------+-----+



In [13]:
# People with stroke based on work type
spark.sql("""SELECT work_type, count(work_type) as work_type_count FROM table 
        WHERE stroke == 1 GROUP BY work_type ORDER BY work_type_count DESC""").show()

+-------------+---------------+
|    work_type|work_type_count|
+-------------+---------------+
|      Private|            441|
|Self-employed|            251|
|     Govt_job|             89|
|     children|              2|
+-------------+---------------+



In [14]:
# Checking gender
spark.sql("""SELECT gender, count(gender) as count_gender, 
count(gender)*100/sum(count(gender)) over() as percent FROM table GROUP BY gender""").show()

+------+------------+-------------------+
|gender|count_gender|            percent|
+------+------------+-------------------+
|Female|       25665|  59.13594470046083|
| Other|          11|0.02534562211981567|
|  Male|       17724|  40.83870967741935|
+------+------------+-------------------+



In [15]:
# Percent of males with stroke
spark.sql("""SELECT gender, count(gender), (COUNT(gender) * 100.0) /(SELECT count(gender)
          FROM table WHERE gender == 'Male') as percentage FROM table 
          WHERE stroke = '1' and gender = 'Male' GROUP BY gender""").show()

+------+-------------+----------------+
|gender|count(gender)|      percentage|
+------+-------------+----------------+
|  Male|          352|1.98600767321146|
+------+-------------+----------------+



In [16]:
# Percentage of females
spark.sql("""SELECT gender, count(gender), (COUNT(gender) * 100.0) /(SELECT count(gender) 
FROM table WHERE gender == 'Female') as percentage FROM table 
WHERE stroke = '1' and gender = 'Female' GROUP BY gender""").show()

+------+-------------+----------------+
|gender|count(gender)|      percentage|
+------+-------------+----------------+
|Female|          431|1.67932982661212|
+------+-------------+----------------+



In [17]:
# No. of strokes based on age
spark.sql("""SELECT age, count(age) as age_count FROM table 
WHERE stroke == 1 GROUP BY age ORDER BY age_count DESC""").show()

+----+---------+
| age|age_count|
+----+---------+
|79.0|       70|
|78.0|       57|
|80.0|       49|
|81.0|       43|
|82.0|       36|
|70.0|       25|
|77.0|       24|
|74.0|       24|
|76.0|       24|
|67.0|       23|
|75.0|       23|
|72.0|       21|
|69.0|       20|
|59.0|       20|
|68.0|       20|
|71.0|       19|
|57.0|       19|
|63.0|       18|
|65.0|       18|
|66.0|       17|
+----+---------+
only showing top 20 rows



In [18]:
# Count for people with stroke over 50 years of age
train.filter((train['stroke'] == 1) & (train['age'] > '50')).count()

708

In [21]:
# fill in missing values
train_f = train.na.fill('No Info', subset=['smoking_status'])

In [22]:
# fill in miss values with mean
from pyspark.sql.functions import mean
mean = train_f.select(mean(train_f['bmi'])).collect()
mean_bmi = mean[0][0]
train_f = train_f.na.fill(mean_bmi,['bmi'])

In [23]:
# Onehot encoding for categorical data

from pyspark.ml.feature import (VectorAssembler,OneHotEncoder,
                                StringIndexer)
gender_indexer = StringIndexer(inputCol = 'gender', outputCol = 'genderIndex')
gender_encoder = OneHotEncoder(inputCol='genderIndex', outputCol='genderVec')
ever_married_indexer = StringIndexer(inputCol = 'ever_married', outputCol = 'ever_marriedIndex')
ever_married_encoder = OneHotEncoder(inputCol='ever_marriedIndex', outputCol='ever_marriedVec')
work_type_indexer = StringIndexer(inputCol = 'work_type', outputCol = 'work_typeIndex')
work_type_encoder = OneHotEncoder(inputCol='work_typeIndex', outputCol='work_typeVec')
Residence_type_indexer = StringIndexer(inputCol = 'Residence_type', outputCol = 'Residence_typeIndex')
Residence_type_encoder = OneHotEncoder(inputCol='Residence_typeIndex', outputCol='Residence_typeVec')
smoking_status_indexer = StringIndexer(inputCol = 'smoking_status', outputCol = 'smoking_statusIndex')
smoking_status_encoder = OneHotEncoder(inputCol='smoking_statusIndex', outputCol='smoking_statusVec')

In [24]:
# Creating pipeline for processing data

assembler = VectorAssembler(inputCols=['genderVec',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedVec',
 'work_typeVec',
 'Residence_typeVec',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVec'],outputCol='features')

from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[gender_indexer, ever_married_indexer, work_type_indexer, Residence_type_indexer,
                           smoking_status_indexer, gender_encoder, ever_married_encoder, work_type_encoder,
                           Residence_type_encoder, smoking_status_encoder, assembler, dtc])

In [25]:
# splitting Training & test data
train_data,test_data = train_f.randomSplit([0.7,0.3])

In [26]:
# Creating model & predicting for test data
model = pipeline.fit(train_data)
dtc_predictions = model.transform(test_data)

In [27]:
# Checking accuracy of model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="stroke", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
print('A Decision Tree algorithm had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))

A Decision Tree algorithm had an accuracy of: 98.15%
